# PRÁCTICA 1 BaseV1- ALGORITMO BASADOS EN ENTORNOS Y TRAYECTORIAS

En este fichero realizaremos las funciones base de la práctica

### Lectura de archivos csv
* Matriz delta: La primera fila representa los indices, la segunda es el estado inicial al comienzo del estudio (NO EL NÚMERO DE SLOTS DE CADA ESTACION). Las siguientes representan los movimientos realizados en las estaciones
* Distancias: Para elegir la estación más cercana hay dos matrices:
    * Cercanas_indices: Matriz con las estaciones más cercans a cada índice. Están ordenadas de izquierda a derecha. Es decir, para la estación 0, la estación más cercana es la estación 7,y luego la 4, 6, 10, etc
    * Cercanas_kms: distancia de cada una. Una vez localizamos la estación deseada en la matriz de índices. Buscamos su posición homóloga


In [1]:
import numpy as np
import pandas as pd

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix

### Transformación de los datos delta
En el fichero deltas_5m.csv se encuentra el número inicial de slots ocupados por la mañana (primera fila de la matriz), y todos los movimientos medidos cada hora a los largo del día (a partir de la segunda fila en adelante). Una fila tiene la siguiente estructura:
	
	[0	0	-1	0	0	0	0	0	0	2	0	1	0	0	0	0]
    
Donde su posición en el array indica la estación medida y el valor el número de bicicletas desplazadas. Ejemplos:
* Si el valor es 0, significa que no ha habido ningún moviemiento en dicha estación
* Si el valor es 2, significa que se han dejado 2 biciclestas en dicha estación
* Si el valor es -1, significa que han retirado 1 una bicicleta en dicha estación

In [2]:
def movements_to_list(deltas_df):
    """
    Devuelve array con todos los movimientos realizados en las estaciones. Cada posicion del array esta compuesto por el id de la estacion y la cantidad de bicis retiradas/aniadidas en cada movimiento
    
    Parametros
        deltas_df : DataFrame
            Pandas DataFrame a convertir
    Return
        Numpy array
    """
    move_list = np.array([]) # Inicializamos el array
    rows = len(deltas_df.index) # Mediciones totales realizadas
    columns = len(deltas_df.columns) # Numero de estaciones
    
    # Recorreremos todas las mediciones e iremos transformando todas ellas que sean distintas de 0
    for i in range(0, rows):
        if i != 0:
            row = deltas_df.iloc[i] # Seleccionamos una medicion
            selec = row[row != 0] # Seleccionamos que estaciones han sufrido algun cambio
            for j in range(0, len(selec)):
                lst = np.array([[int(selec.index[j]), selec[j]]]) # Obtenemos un array con el index y el numero de bicis retiradas/aniadidas
                # En la setencia anterior, el primer corchete sirve para crear el array. Mientras que el segundo para crear una tupla tal que: [estacion, movements]
                if len(move_list) == 0:
                    move_list = lst # Asignamos la primera tupla como 
                else:
                    move_list = np.append(move_list, lst, axis=0) # Aniadimos una nueva fila(axis=0) a move_list
    
    return move_list

## Función de evaluación

In [3]:
def evaluate(move_list, init_state, capacity, index_m, kms_m):
    '''
    La funcion estipula el numero de kilometros recorrido por los usuarios al no encontrar una bicicleta en su estación de preferencia y tener que desplazarse a la estaion mas cercana con bicicletas disponibles, o la distancia recorrida en caso de no haber sitio para dejar la bicicleta en la estación deseada y tener que desplazarse a la estacion mas cercana con capacidad. La distancia realizada a pie es 3 veces mayor que la recorrida en bicicleta
    
    Parametros
        move_list : Numpy ndarray
            lista con todos los movimientos realizados. Cada elemento de la lista debe tener la siguiente estructura: [index, desplazamientos]
        init_state : Numpy ndarray
            Contiene el numero de biciclestas alojadas en cada estacion al comienzo de la evaluacion
        capacity : Numpy ndarray
            Contiene la capacidad maxima de bicicletas que puede almacenar una estacion
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
    Returns
        El numero total de kilometros recorridos por los usuarios
    '''
    tkms = 0 # Kilometros totales
    travel_kms = 0 # Kilometros recorridos por un usuario
    actual_state = init_state.copy() # Variable en la que iremos modificando el numero de biciclestas que se encuentran en cada estacion
    walk_multiplier = 3 # La distancia recorrida andando costara 3 veces mas que la recorrida en bici (valor calculado en kms_m es la distancia en bici)
    
    for move in move_list:
        station = move[0] # Guardamos la estacion 
        n_bicycle = actual_state[station] + move[1]  # Al numero de bicis de la estacion es cuestion, se le suma el numero de bicis desplazadas (este valor puede ser positivo o negativo) 
        
        # Tenemos que comprobar que el numero de desplazamientos es posible. Es decir, en caso de que se hayan retirado biciletas, que hubiese suficientes para suplir la demanda.
        # Y caso de que se quisiera dejar bicicletas, que existan suficientes slots/espaciones disponibles en la estacion
        if n_bicycle >= 0 and n_bicycle <= capacity[station]: 
            actual_state[station] = n_bicycle # Como el numero obtenido es posible, actualizamos el estado actual de la estacion
        else:
            # En caso de que no existan bicicletas suficientes para suplir la demanda en la estacion
            if n_bicycle < 0:
                actual_state[station] = 0 # La estacion ahora tiene 0 bicicletas
                search = abs(n_bicycle) # abs() nos devuelve el valor absoluto. Obtenemos el valor de bicicletas que necesitamos buscar
                nearest_station_index = find_nearest_station_index(index_m, station, actual_state, capacity, search, 'bicycles') # Obtenemos la estacion mas cercana con capacidad suficiente de bicicletas
                
                # Calculamos los kms recorridos para llegar a la estacion mas cercana
                travel_kms = kms_m[station][nearest_station_index] * search * walk_multiplier # Kms hacia la estacion * numero de bicicletas a buscar * multiplicador por andar
                
                # Actualizamos el estado de la estacion mas cercana
                nearest_station = index_m[station][nearest_station_index] # Estacion mas cercana a la nuestra
                actual_state[nearest_station] -= search # Le restamos el numero de bicicletas necesario
            
            # En caso de que no existan slots suficientes para suplir la demanda en la estacion
            elif n_bicycle > capacity[station]:
                actual_state[station] = capacity[station] # Ahora la estacion esta llena
                search = n_bicycle - capacity[station] # Obtenemos el numero de slots a buscar
                nearest_station_index = find_nearest_station_index(index_m, station, actual_state, capacity, search, 'slots') # Obtenemos la estacion mas cercana con capacidad suficiente de slots
                
                # Calculamos los kms recorridos para llegar a la estacion mas cercana
                travel_kms = kms_m[station][nearest_station_index] * search # Kms hacia la estacion * numero de slots a buscar
                
                # Actualizamos el estado de la estacion mas cercana
                nearest_station = index_m[station][nearest_station_index] # Estacion mas cercana a la nuestra
                actual_state[nearest_station] += search # Le sumamos el numero de bicicletas dejas en los slots libres
                
            tkms += travel_kms # Sumamos los kilometros recorridos al total
            
    return tkms

def find_nearest_station_index(index_m, station, actual_state, capacity, search, search_type):
    """
    Devuelve la posicion (index-columna) de la estacion mas cercana con respecto al parametro station en la variable index_m
    
    Parametros
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        station : int
            Estacion desde la que buscar su estacion vecina mas proxima
        actual_state : Numpy ndarray
            Array con el estado actual de bicicletas en cada estacion
        capacity : Numpy ndarray
            Contiene la capacidad maxima de bicicletas que puede almacenar una estacion
        search : int
            Numero de bicicletas o slots a buscar
        search_type : str
            Cadena de caracteres que nos indica el tipo de elemento a buscar: 'bicycles' o 'slots'
            
    Return
        Devuelve un int
    """
    # Valor que almacena la posicion (columna) de la estacion mas cercana con respecto al parametro station
    row_selec = index_m[station] # Seleccionamos el array de estaciones cercanas de la estacion station
    
    # Comprobamos que tipo de busqueda estamos realizando, 'bicycles' o 'slots'
    if search_type == 'bicycles':
        # Recorremos el array seleccionado y comprobamos cual es la estacion mas cercana con bicicletas suficientes
        for j in range(1, len(row_selec)):
            # Seleccionamos con row_selec la estacion mas cercana a station, despues comprobamos la capacidad disponibles. Y por ultimo comprobamos si es suficiente
            if search <= actual_state[row_selec[j]]:
                return j # Devolvemos el index
            
    elif search_type == 'slots':
        # Recorremos el array seleccionado y comprobamos cual es la estacion mas cercana con slots suficientes
        for j in range(1, len(row_selec)):
            # Seleccionamos con row_selec la estacion mas cercana a station, despues comprobamos la capacidad disponibles. Y por ultimo comprobamos si es suficiente
            free_slots = capacity[row_selec[j]] - actual_state[row_selec[j]] # Calculamos el numero de slots disponibles restandole a la capacidad total el numero de bicicletas actuales
            if search <= free_slots:
                return j # Devolvemos el index
    else:
        raise Exception("La variable search_type no contiene un tipo valido") # La palabra 'raise' nos permite lanzar una excepcion. Es igual que la palabra 'throw' en otros lenguajes
        

def show_variation(init_solution, init_tkms, new_solution, new_tkms):
    """
    Muestra por pantalla la diferencia de slots existentes entre la ambas soluciones
    
    Parametros
        init_solution : Numpy ndarray
            Solucion inicial a comparar
        new_solution : Numpy ndarray
            Solucion a comparar con respecto a init_solution
    """
    
    columns = np.arange(0,len(init_solution))
    variation = abs(init_solution - best_solution)
    
    # Para representarlo carrectamente la variacion crearemos un string con los espacios correspondientes
    string = '['
    space = ' '
    for i in range(0,len(init_solution)):
        if variation[i] < 10:
            string += space + str(variation[i])
        else:
            string += str(variation[i])
        
        if i != len(init_solution)-1:
            string += space
        
    string += ']'
    
    print('----------------------COMPARACION DE SOLUCIONES-----------------------')
    print('    Estacion    : {}'.format(columns))
    print('----------------------------------------------------------------------')
    print('Solucion inicial: {} - {} km'.format(init_solution, init_tkms))
    print(' Solucion final : {} - {} km'.format(new_solution, new_tkms))
    print('----------------------------------------------------------------------')
    print('    Variacion   : ' + string)
    print('----------------------------------------------------------------------')

## Generación de la solución inicial

In [4]:
def generate_initial_solution(init_state, max_slots, seed):
    '''
    Crea una solucion a partir del estado inicial de las estaciones medidas y el numero de slots disponibles
    
    Parametros
        init_state : Numpy ndarray
            Estado inicial de las estaciones
        max_slots : int
            Numero maximo de slots disponibles a repartir
        seed : int
            Semilla con la que generar numeros aleatorios
    
    Return
        Devuelve un array con el numero de slots disponibles en cada estacion (capacity)
    '''
    np.random.seed(seed) # Inicializamos la semilla
    init_solution = init_state.copy() # Nuestro punto de partida para crear nuestra solucion sera el estado inicial de las estaciones
    taken_slots = init_solution.sum() # Numero de slots ocupados inicialmente
    
    while taken_slots < max_slots:
        selec = np.random.randint(0,len(init_state)) # Seleccionamos una estacion de forma aleatoria
        init_solution[selec] += 1 # Aumentamos el numero de slots disponibles en 1
        taken_slots += 1 # Aumentamos el numero de slots disponibles
    
    return init_solution
    

## Operador de moviemiento

Para crear el operador necesitaremos tener un registro de los movimientos explorados. Para ello, crearemos una matriz en la que comprobaremos los movimientos realizados. Crearemos una matriz escalonada con la siguiente estructura:
    
   
    Estaciones|   0   1   2   3   4   5   6   7   8 
    ------------------------------------------------
         0    | [-1   0   0   0   0   0   0   0   0]
         1    | [-1  -1   0   0   0   0   0   0   0]
         2    | [-1  -1  -1   0   0   0   0   0   0]
         3    | [-1  -1  -1  -1   0   0   0   0   0]
         4    | [-1  -1  -1  -1  -1   0   0   0   0]
         5    | [-1  -1  -1  -1  -1  -1   0   0   0]
         6    | [-1  -1  -1  -1  -1  -1  -1   0   0]
         7    | [-1  -1  -1  -1  -1  -1  -1  -1   0]
         8    | [-1  -1  -1  -1  -1  -1  -1  -1  -1]
            
En la que las celdas con valoro '-1' representan valores que no se deben explorar. Esto es así para no repetir movimientos entre estaciones previamente seleccionadas o realizar movimientos imposibles. Por ejemplo: 
    
    * Celda (0,0): No podemos mover realizar un movimiento de slots de la estacion 0 a la estacion 0. Este mismo ejemplo se repite en toda la diagonal
    
    * Celda (2,1): Esta tiene el valor '-1' porque en la celda (1,2) ya explora un movimiento entre estas dos estaciones

Por último, cabe aclarar que marcaremos un movimiento explorado con el valor '1'

In [5]:
def movement_operator(neighbours_m, init_state, actual_solution, n_slots):
    """
    Funcion que nos devuelve una nueva solucion sin explorar a partir de la actual.
    En caso de haberse explorado todos los movimientos, la funcion lanzara un error adviertiendo el problema
    
    Parametros
        neighbours_m : Numpy ndarray
            Matriz que contiene los movimientos explorados
        actual_solution : Numpy ndarray
            Array que contiene la solucion actual
        n_slots : int
            Cantidad de slots a mover con el movimiento
        
    Return
        Devuelve una tupla con la nueva solucion calculada mas la matriz neighbours_m actualizada
        Ejemplo: (new_solution, new_neighbours_m)
    """
    new_solution = actual_solution.copy()
    new_neighbours_m = neighbours_m.copy()
    rows, columns = neighbours_m.shape # Guardamos la estructura de la matriz
    
    # Buscamos el primer movimiento que no se haya realizado. Para ello, recorreremos la matriz neighbours_m en busca de la primera celda sin explorar que encontremos (celda = 0)
    for i in range(0, rows):
        for j in range(i+1,columns): # Nos posicionamos en la columna siguiente al ultimo '-1' de la fila
            # Buscamos el primer movimiento sin explorar
            if neighbours_m[i,j] == 0: 
                # Ahora cambiamos n_slots entra las estaciones 'i' y 'j' si es posible realizar el cambio (la capacidad de una estacion no puede ser negativa). Y ademas, comprobamos si el cambio es compatible con el estado inicial de la estacion
                if new_solution[i] >= n_slots and (new_solution[i]-n_slots) >= init_state[i]:
                    new_solution[i] -= n_slots # Le quitamos n_slots a la estacion 'i'
                    new_solution[j] += n_slots # Le aniadimos los n_slots quitados de 'i' a 'j'

                    # Actualizamos la matriz de vecinos
                    new_neighbours_m[i,j] = 1 # Indicamos que se ha explorado este movimiento

                    return [new_solution, new_neighbours_m]
                else:
                    new_neighbours_m[i,j] = 1 # Indicamos que se ha explorado este movimiento
                    
    
    raise Exception("Se han explorado todos los movimientos posibles") # La palabra 'raise' nos permite lanzar una excepcion. Es igual que la palabra 'throw' en otros lenguajes
    

def init_neighbours_matrix(n_station):
    """
    Devuelve una matriz con los vecinos a explorar. Cada posicion de las filas representa una estacion, y de igual forma con cada una de las columnas. Las posiciones con valor a '-1' representan celdas que no se deben explorar. Matriz de ejemplo :
         
         ----------------------------------
         |Estacion| 0 | 1 | 2 | 3 | 4 | 5 |
         |---------------------------------
         |    0   |-1 | 0 | 0 | 0 | 0 | 0 | 
         |    1   |-1 |-1 | 0 | 0 | 0 | 0 |
         |    2   |-1 |-1 |-1 | 0 | 0 | 0 | 
         |    3   |-1 |-1 |-1 |-1 | 0 | 0 |
         |    4   |-1 |-1 |-1 |-1 |-1 | 0 |
         |    5   |-1 |-1 |-1 |-1 |-1 |-1 |
         ----------------------------------
    
    Parametros
        n_station : int
            Numero de estaciones
    
    Return
        Devuelve un numpy ndarray 
    """
    neighbours_m = np.eye(n_station) * -1 # Creamos una matriz intendidad con una diagonal de valor -1
    
    # Recorremos la matriz y rellenamos la parte izquierda con -1
    for row in neighbours_m:
        for j in range(0,len(row)):
            if row[j] != -1:
                row[j] = -1
            else:
                break
                
    return neighbours_m

# Pruebas

In [7]:
import numpy as np
import pandas as pd

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m.csv')

index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix

seed = 1 # Semilla inicial
n_slots = 4 # Numero de slots a mover en cada movimiento
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial
init_solution = generate_initial_solution(init_state, 220, seed) # Generamos la solucion inicial
move_list = movements_to_list(deltas_df) # Creamos la lista de movimientos


neighbours_m = init_neighbours_matrix(len(init_state)) # Creamos la matriz de movimientos vecinos
actual_solution = init_solution.copy()
init_tkms = evaluate(move_list, init_state, init_solution, index_m, kms_m) # Evaluamos nuestra solucion
best_solution = actual_solution.copy()
best_tkms = init_tkms

i = 0 # Contador de soluciones
print('Init_solution: {} - {} kms'.format(init_solution, best_tkms))

try:
    while True: # Realizaremos el bucle mientras haya movimientos a explorar
        actual_solution, neighbours_m = movement_operator(neighbours_m, init_state, best_solution, n_slots) # Generamos una nueva solucion
        new_tkms = evaluate(move_list, init_state, actual_solution, index_m, kms_m) # Evaluamos nuestra solucion
        if best_tkms > new_tkms:
            best_solution = actual_solution.copy()
            best_tkms = new_tkms
            print('New best_solution({}): {} - {} kms '.format(i, best_solution, best_tkms))
        
        i += 1
            
except Exception as e:
    print(e) # Realizaremos el bucle mientras haya movimientos a explorar

print('')
show_variation(init_solution, init_tkms, best_solution, best_tkms)

Init_solution: [ 9 11 16  7 11 16 11 14  9 16 11 21 14 18 18 18] - 69.51756524864952 kms
New best_solution(1): [ 5 11 20  7 11 16 11 14  9 16 11 21 14 18 18 18] - 67.8167596691798 kms 
New best_solution(2): [ 5  7 24  7 11 16 11 14  9 16 11 21 14 18 18 18] - 66.46378662442508 kms 
New best_solution(3): [ 5  7 20 11 11 16 11 14  9 16 11 21 14 18 18 18] - 61.4838383374686 kms 
New best_solution(11): [ 5  7 16 11 11 16 11 14  9 16 11 25 14 18 18 18] - 60.09093178621756 kms 
New best_solution(46): [ 5  7 16 11 11 16 11 14  9 16 11 25 14 14 18 22] - 56.189881322013555 kms 
Se han explorado todos los movimientos posibles

----------------------COMPARACION DE SOLUCIONES-----------------------
    Estacion    : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
----------------------------------------------------------------------
Solucion inicial: [ 9 11 16  7 11 16 11 14  9 16 11 21 14 18 18 18] - 69.51756524864952 km
 Solucion final : [ 5  7 16 11 11 16 11 14  9 16 11 25 14 14 18 22] - 56.18